In [ ]:
import pandas as pd
from sqlalchemy import create_engine
    
engine = create_engine("sqlite:///mta_data.db")
df21 = pd.read_sql("SELECT * FROM mta_data_H1_19to21;", engine)

df = df21.copy()

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df.info()
df.columns
df.head(5)
df.value_counts()
df.groupby("STATION")["DT"].count()
df.TIME.value_counts().head(30)
df.TIME.value_counts().tail(30)

In [ ]:
import datetime as dt
df["DATE_TIME"] = pd.to_datetime(df.DT + " " + df.TIME, format = "%m/%d/%Y %H:%M:%S") #CONVERT TO DATETIME

In [ ]:
(df
.groupby(["CA","UNIT","SCP","STATION","DATE_TIME"])
.ENTRIES.count()
.reset_index()
.sort_values("ENTRIES",ascending=False)).head(7) 

In [ ]:
df.sort_values(["CA", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True, ascending=True)
df.sort_values(["DATE_TIME", "DESC"], inplace=True, ascending=[True,False])

df.drop_duplicates(subset=["CA", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)

In [ ]:
df.drop(["EXITS"], axis=1, inplace=True, errors="ignore")

In [ ]:
df.shape

In [ ]:
df_4hr = df.copy()
df_4hr[["PREV_DATE","PREV_TIME","PREV_DATETIME","PREV_ENTRIES"]] = (df_4hr.groupby(["CA", "UNIT", "SCP", "STATION"])[["DT","TIME","DATE_TIME","ENTRIES"]].shift(1))
df_4hr.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

In [ ]:
(df_4hr["ENTRIES"] - df_4hr["PREV_ENTRIES"]).describe()

In [ ]:
df_4hr[df_4hr["ENTRIES"] < df_4hr["PREV_ENTRIES"]].shape

In [ ]:
(df_4hr[df_4hr["ENTRIES"] < df_4hr["PREV_ENTRIES"]]
 .groupby(["CA","UNIT","SCP","STATION"])
 .size())

In [ ]:
def get_daily_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    
    if counter < 0:
        counter = -counter
    
    if counter > max_counter:
        print(f'entries: {row["ENTRIES"]} <-- {row["PREV_ENTRIES"]}')
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
    
    if counter > max_counter:
        return 0
    
    return counter

df_4hr["DAILY_ENTRIES"] = df_4hr.apply(get_daily_counts, axis=1, max_counter=23040)

In [ ]:
df_4hr["DAILY_ENTRIES_mean"] = (df_4hr.groupby(["CA","UNIT","SCP","STATION"]).DAILY_ENTRIES.transform(lambda x: x.mean()))
df_4hr["DAILY_ENTRIES_std"] = (df_4hr.groupby(["CA","UNIT","SCP","STATION"]).DAILY_ENTRIES.transform(lambda x: (x - x.mean())/x.std()))

df4_n = df_4hr[df_4hr["DAILY_ENTRIES_std"] <= 2]

In [ ]:
df4_n.drop(["DAILY_ENTRIES_mean"], axis=1, inplace=True, errors="ignore")
df4_n.drop(["DAILY_ENTRIES_std"], axis=1, inplace=True, errors="ignore")
df4_n.drop(["PREV_DATE"], axis=1, inplace=True, errors="ignore")
df4_n.drop(["PREV_TIME"], axis=1, inplace=True, errors="ignore")
df4_n.drop(["PREV_ENTRIES"], axis=1, inplace=True, errors="ignore")

In [ ]:
#dftoi = df_4hr.set_index("DATE_TIME").between_time('9:59:00','16:01:00')
df4_n["T"] = pd.to_datetime(df4_n["TIME"], format = "%H:%M:%S").dt.time

In [ ]:
mask = ((df4_n["T"] >= dt.time(10,0,0)) & (df4_n["T"] <= dt.time(16,0,0)))
dftoi = df4_n[mask]

In [ ]:
topstations_2021 = df_4hr[(df_4hr.DATE_TIME >= "2021-01-01 00:00:00") & (df_4hr.DATE_TIME < '2021-07-31 23:59:59')]

In [ ]:
topstations_2021_g = topstations_2021.groupby(["STATION"]).DAILY_ENTRIES.sum().reset_index().sort_values("DAILY_ENTRIES", ascending=False)

In [ ]:
topstations_2021_g.head(20)

In [ ]:
df4_n["DoW"] = df4_n["DATE_TIME"].dt.dayofweek
DoW = {0: "Mon", 1: "Tue", 2: "Wed", 3: "Thu", 4: "Fri", 5: "Sat", 6: 'Sun'}
df4_n["DoW_n"] = df4_n.DoW.map(DoW)
df4_n["H"] = df4_n["DATE_TIME"].dt.hour

In [ ]:
df4_p = df4_n.groupby(["H","DoW_n"]).DAILY_ENTRIES.median().reset_index()

In [ ]:
df4_p.head(20)

In [ ]:
df4_pv = df4_p.pivot(index='H', columns='DoW_n', values='DAILY_ENTRIES')
column_order = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
df4_pvt = df4_pv.reindex(column_order, axis=1)
df4_pvt

In [ ]:
sns.heatmap(df4_pvt,cmap='RdBu_r',annot=True, fmt="g")

sns.set(rc = {'figure.figsize':(15,8)})
plt.title("Median traffic levels at NYC subway turnstiles (entries)", fontsize=20)
plt.xlabel("Day of Week", fontsize=15)
plt.ylabel("Time of Day", fontsize=15);

In [ ]:
mask = ((df4_n["H"] == 9) | (df4_n["H"] == 10) | (df4_n["H"] == 11) |\
        (df4_n["H"] == 13) | (df4_n["H"] == 14) | (df4_n["H"] == 15) |\
        (df4_n["H"] == 16) | (df4_n["H"] == 17) | (df4_n["H"] == 17))
disp_H = df4_n[mask]
disp_HV = disp_H.groupby(["STATION","H"]).DAILY_ENTRIES.sum().reset_index()
disp_HV2 = disp_HV[["H","DAILY_ENTRIES"]]

In [ ]:
sns.violinplot(data=disp_HV2, x="H", y="DAILY_ENTRIES", fmt="g");
plt.title("Hourly traffic (entry) dispersion", fontsize=20)
plt.xlabel("Hour of Day", fontsize=15)
plt.ylabel("Entries per Station", fontsize=15);

In [ ]:
dfpenn = df4_n[(df4_n["STATION"] == "34 ST-PENN STA") | (df4_n["STATION"] == "34 ST-HERALD SQ")]
dfpenn.CA.unique()

In [ ]:
dfpennm = dfpenn.groupby("CA").sum().reset_index().sort_values("DAILY_ENTRIES", ascending=False)

In [ ]:
dfpennstation = dfpenn.groupby(["STATION","CA"]).size().reset_index().rename(columns={0:"count"})
dfpennstation.drop("count", axis=1)
dfpennm = pd.merge(dfpennm, dfpennstation, how="left", on=["CA"])

In [ ]:
dfpennm.head()

In [ ]:
sns.barplot(data=dfpennm, x="CA", y="DAILY_ENTRIES")
plt.title("Traffic dispersion by Control Area at the top 2 stations, Penn Station & 34th St. Herald Square", fontsize=20)
plt.xlabel("Control Area", fontsize=15)
plt.ylabel("Cumulative entry traffic (Jan to Jul, 2019 to 2021)", fontsize=15)
plt.ticklabel_format(style='plain', axis='y')

In [ ]:
mask = (((df4_n["DoW_n"] != "Sat") | (df4_n["DoW_n"] != "Sun")) & ((df4_n["H"] >= 10) & (df4_n["H"] <= 16)))
dfts = df4_n[mask].copy()

In [ ]:
dfts["std"] = (dfts.groupby(["STATION","CA","DoW_n","H"]).DAILY_ENTRIES.transform(lambda x: (x.std())))

In [ ]:
dfts["range"] = (dfts.groupby(["STATION","CA","DoW_n","H"]).DAILY_ENTRIES.transform(lambda x: (x.max() - x.min())))

In [ ]:
dfts_m = dfts.groupby(["STATION","CA","DoW_n","H"])[["DAILY_ENTRIES"]].median().reset_index()
dfts_m.sort_values("DAILY_ENTRIES", ascending=False, inplace=True)

In [ ]:
dfts_m_chart = dfts_m[0:50]
dfts_m_chart

In [ ]:
dfts_m_chart = pd.merge(dfts_m_chart, dfts[["CA","DoW_n","H","range","std"]], how="left", on=["CA","DoW_n","H"])

dfts_m_chart.head(10)

In [ ]:
dfts_m_chart.drop_duplicates(subset=["CA", "STATION", "DoW_n", "H"], inplace=True)

In [ ]:
dfts_m_chart.head(20)

In [ ]:
dfts_m_chart.dropna(subset=["std"], axis=0, inplace=True)
dfts_m_chart.reset_index()
dfts_m_chart["std"] = dfts_m_chart["std"].round(1)

In [ ]:
dfts_m_chart.sort_values(["STATION","DAILY_ENTRIES","std"], ascending=[True,False,True])
dfts_m_chart.rename(columns={"DAILY_ENTRIES":"Median", "std":"Standard Deviation", "range":"Range", "DoW_n":"Day of Week", "H":"Hour of Day"}, inplace=True)

In [ ]:
dfts_m_chart["Rank, by Median"] = range(1,len(dfts_m_chart["Median"])+1)

In [ ]:
dfts_m_chart.sort_values(["STATION","Median","Standard Deviation"], ascending=[True,False,True])

In [ ]:
dftsmchart = dfts_m_chart[(dfts_m_chart["Rank, by Median"]<=30)].head(30)

DoW1 = {"Mon":0, "Tue":1, "Wed":2, "Thu":3, "Fri":4, "Sat":5, 'Sun':6}
dftsmchart["DOW"] = dftsmchart["Day of Week"].map(DoW1)
dftsmchart.sort_values(["DOW", "Hour of Day", "Rank, by Median"], ascending=True, inplace=True)
dftsmchart.drop("DOW", axis=1, inplace=True)
dftsmchart.head(30)

In [ ]:
dftoi["Year"] = dftoi.DATE_TIME.dt.year

In [ ]:
dftoiy = dftoi.drop(["DIVISION","TIME","DESC","ENTRIES","PREV_DATETIME","T"], axis=1)

In [ ]:
dftoiy["daynum"] = dftoiy["DATE_TIME"].dt.dayofyear

In [ ]:
dftoiy19 = dftoiy[dftoiy["Year"] == 2019]
dftoiy19s = dftoiy19.sort_values("daynum",ascending=True)
dftoiy19m = dftoiy19s.groupby("daynum").DAILY_ENTRIES.median().reset_index()
dftoiy20 = dftoiy[dftoiy["Year"] == 2020]
dftoiy20s = dftoiy20.sort_values("daynum",ascending=True)
dftoiy20m = dftoiy20s.groupby("daynum").DAILY_ENTRIES.median().reset_index()
dftoiy21 = dftoiy[dftoiy["Year"] == 2021]
dftoiy21s = dftoiy21.sort_values("daynum",ascending=True)
dftoiy21m = dftoiy21s.groupby("daynum").DAILY_ENTRIES.median().reset_index()

In [ ]:
from functools import reduce
dftoiy_all = [dftoiy19m, dftoiy20m, dftoiy21m]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['daynum'], how='inner'), dftoiy_all)

In [ ]:
df_merged.rename(columns={"DAILY_ENTRIES_x":"Median Traffic 2019", "DAILY_ENTRIES_y": "Median Traffic 2020", "DAILY_ENTRIES": "Median Traffic 2021"})

In [ ]:
from matplotlib import pylab

#sns.lineplot(data=df_merged, x="daynum")
sns.lineplot(data=dftoiy19m, x="daynum", y="DAILY_ENTRIES")

z19 = np.polyfit(dftoiy19m.daynum, dftoiy19m.DAILY_ENTRIES, 1)
p19 = np.poly1d(z19)
pylab.plot(dftoiy19m.daynum,p19(dftoiy19m.daynum),"b--")

sns.lineplot(data=dftoiy20m, x="daynum", y="DAILY_ENTRIES")

#z20 = np.polyfit(dftoiy20m.daynum, dftoiy20m.DAILY_ENTRIES, 1)
#p20 = np.poly1d(z20)
#pylab.plot(dftoiy20m.daynum,p20(dftoiy20m.daynum))

sns.lineplot(data=dftoiy21m, x="daynum", y="DAILY_ENTRIES")

z21 = np.polyfit(dftoiy21m.daynum, dftoiy21m.DAILY_ENTRIES, 1)
p21 = np.poly1d(z21)
pylab.plot(dftoiy21m.daynum,p21(dftoiy21m.daynum),"g--")

plt.legend(['2019','2020', '2021'], shadow = True);
plt.xticks(ticks=list(range(0,212,30)));
plt.title("Daily entry traffic per turnstile from 2019 to 2020", fontsize=20)
plt.xlabel("Days from January 1", fontsize=12)
plt.ylabel("Median entry traffic at each turnstile", fontsize=12)